In [1]:
from pytube import YouTube
from youtube_search import YoutubeSearch
import csv
import os
import re
from datetime import datetime

csv.register_dialect('mydialect', delimiter=';', quoting=csv.QUOTE_ALL)

max_videos_per_song = 5

def sanitize_filename(filename):
    return re.sub(r'[\\/*?:"<>|]', '', filename)

def search_video(artist, song):
    results = YoutubeSearch(f'{artist} {song}', max_results=10).to_dict()
    links = []
    for result in results:
        links.append('https://www.youtube.com' + result['url_suffix'])
    return links

def download_video(url, year, ranking, index):
    youtubeObject = YouTube(url)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    resolution = youtubeObject.resolution
    filename = youtubeObject.default_filename
    filename = filename.replace('.mp4', f' {index} {resolution}.mp4')
    filename = sanitize_filename(filename)

    if os.path.exists(f'{year}/{ranking}/{filename}'):
        return filename
    else:
        youtubeObject.download(f'{year}/{ranking}', filename=filename)
        return filename

def download_data(year, start, end):    
    with open('list.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f, dialect='mydialect')
        for row in reader:
            if len(row) > 0:
                ranking = int(row[1])
                if row[0] == year and ranking >= start and ranking <= end:
                    
                    artist = row[2]
                    song = row[3]

                    if not os.path.exists(f'{year}/{ranking}'):
                        os.makedirs(f'{year}/{ranking}')

                    file = sanitize_filename(f'{artist} {song}')
                    open(f'{year}/{ranking}/{file}', 'a').close()
                    
                    links = search_video(artist, song)

                    count = 0
                    for index, link in enumerate(links):
                        if count < max_videos_per_song:
                            try:
                                filename = download_video(link, year, ranking, index)
                                print(f"{datetime.now()} Download is completed successfully {year} {ranking} : {filename}")
                                count += 1
                            except:
                                print(f"{datetime.now()} An error has occurred - {link}")


download_data('2022', 1, 50)

2024-01-09 11:58:41.866046 Download is completed successfully 2022 44 : Nukkuvat aamut 0 720p.mp4
2024-01-09 11:58:42.978569 Download is completed successfully 2022 44 : Nukkuvat aamut - Lauri Tähkä (LYRICS) 1 720p.mp4
2024-01-09 11:58:43.708467 An error has occurred - https://www.youtube.com/watch?v=F9G7xdDkcek&pp=ygUdVMOkaGvDpCwgTGF1cmkgTlVLS1VWQVQgQUFNVVQ%3D
2024-01-09 11:59:03.618666 Download is completed successfully 2022 44 : Lauri Tähkä - Nukkuvat aamut live Suomipop Jyväskylä 2022 3 720p.mp4
2024-01-09 11:59:05.768686 Download is completed successfully 2022 44 : Aavikko 4 720p.mp4
2024-01-09 11:59:11.577413 Download is completed successfully 2022 44 : Väkevänä kuin metsä 5 720p.mp4
2024-01-09 11:59:18.583208 Download is completed successfully 2022 45 : Yksinkertasta 0 720p.mp4
2024-01-09 11:59:22.489605 Download is completed successfully 2022 45 : Ahti - Yksinkertasta (Lyrics) 1 720p.mp4
2024-01-09 11:59:39.082726 Download is completed successfully 2022 45 : Ahti-Yksinkertasta 